In [1]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [2]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [3]:
# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGB')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [4]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [5]:
!pip3 install torch==1.13 torchvision --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip3 install yolox@git+https://github.com/braiansmarzaro/ByteTrack.git loguru thop lap cython_bbox --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 973.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [7]:
!pip3 install cython 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI' --quiet

  Preparing metadata (setup.py) ... done


In [8]:
class ByteTrackerArguments:
    def __init__(self, *args):
        self.track_thresh, self.track_buffer, self.match_thresh = args
        self.mot20 = True

args = ByteTrackerArguments(0.50, 25, 0.80)

In [9]:
from yolox.tracker.byte_tracker import BYTETracker

tracker = BYTETracker(args)

In [10]:
!pip3 install -U openmim --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.1/953.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [11]:
!git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 38019, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 38019 (delta 38), reused 67 (delta 34), pack-reused 37917
Receiving objects: 100% (38019/38019), 63.31 MiB | 16.72 MiB/s, done.
Resolving deltas: 100% (26187/26187), done.


In [12]:
!mim install "mmengine" "mmcv-full" "mmdet" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 MB 8.8 MB/s eta 0:00:00


In [13]:
!wget -O /content/checkpoint.pth https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet_tiny_8xb32-300e_coco/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth

--2024-04-13 19:40:23--  https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet_tiny_8xb32-300e_coco/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.20.217, 47.246.20.218, 47.246.20.225, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.20.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57532893 (55M) [application/octet-stream]
Saving to: ‘/content/checkpoint.pth’

/content/checkpoint 100%[===================>]  54.87M  16.1MB/s    in 4.0s    

2024-04-13 19:40:28 (13.8 MB/s) - ‘/content/checkpoint.pth’ saved [57532893/57532893]



In [19]:
import torch
from mmdet.apis import inference_detector, init_detector

device = "cuda" if torch.cuda.is_available() else "cpu"
model = init_detector("/content/rtmdet.py", "/content/checkpoint.pth", device=device)
js = video_stream()
label_html = 'Capturing...'
image_html = ''

while True:
    js_reply = video_frame(label_html, image_html)
    if not js_reply:
        break

    img = js_to_image(js_reply["img"])

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    result = inference_detector(model, img)

    predicted_bboxes = result.pred_instances.bboxes
    predicted_scores = result.pred_instances.scores
    predicted_labels = result.pred_instances.labels

    filtered_bboxes = []
    filtered_scores = []
    for box, label, score in zip(predicted_bboxes, predicted_labels, predicted_scores):
        if label == 0 and score > 0.60:
            filtered_bboxes.append(box.cpu().numpy())
            filtered_scores.append(score.cpu().numpy())

    max_bbox = None
    max_score = None
    max_area = 0
    for box in filtered_bboxes:
        area = (box[2] - box[0]) * (box[3] - box[1])
        if area > max_area:
            max_bbox = box
            max_area = area
            max_score = filtered_scores[filtered_bboxes.index(box)]


    try:
        dets = [(max_bbox[0], max_bbox[1], max_bbox[2], max_bbox[3], max_score)]
    except:
        print("Could not track the person")
        continue

    online_targets = tracker.update(np.array(dets), img.shape, img.shape)
    tracked_bbox = online_targets[0].tlbr
    tracked_id = online_targets[0].track_id

    if tracked_bbox.shape[0] == 0:
        print("No person was detected")
        continue
    else:
        cv2.rectangle(img, (int(tracked_bbox[0]), int(tracked_bbox[1])), (int(tracked_bbox[2]), int(tracked_bbox[3])), (0, 255, 0), 2)
        cv2.putText(img, f"ID: {tracked_id}", (int(tracked_bbox[0]), int(tracked_bbox[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        image_html = bbox_to_bytes(img)

    image_center = (img.shape[1] / 2, img.shape[0] / 2)
    bbox_center_x = max_bbox[0] + (max_bbox[2] - max_bbox[0]) / 2
    bbox_center_y = max_bbox[1] + (max_bbox[3] - max_bbox[1]) / 2

    message = ""
    if bbox_center_x > image_center[0]:
        message += "\033[92mMove left by {}\033[0m and ".format(round(bbox_center_x - image_center[0], 2))
    elif bbox_center_x < image_center[0]:
        message += "\033[92mMove right by {}\033[0m and ".format(round(image_center[0] - bbox_center_x, 2))
    if bbox_center_y > image_center[1]:
        message += "\033[91mMove up by {}\033[0m".format(round(bbox_center_y - image_center[1], 2))
    elif bbox_center_y < image_center[1]:
        message += "\033[91mMove down by {}\033[0m".format(round(image_center[1] - bbox_center_y, 2))
    print(message)

Loads checkpoint by local backend from path: /content/checkpoint.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



<IPython.core.display.Javascript object>

Could not track the person
Could not track the person
Move right by 0.01 and Move up by 77.56
Move right by 11.34 and Move up by 83.88
Move left by 10.14 and Move down by 3.7
Move left by 91.2 and Move down by 1.66
Move left by 163.48 and Move down by 2.06
Move left by 163.9 and Move down by 2.16
Move left by 124.48 and Move down by 1.7
Move left by 138.67 and Move down by 2.84
Move left by 15.79 and Move up by 9.52
Move right by 90.76 and Move down by 2.19
Move right by 128.62 and Move down by 1.91
Move right by 135.76 and Move up by 1.61
Could not track the person
Move right by 104.58 and Move down by 1.94
Move right by 55.16 and Move down by 0.74
Move right by 87.22 and Move down by 1.41
Move right by 97.35 and Move down by 3.96
Could not track the person
Could not track the person
Could not track the person
Could not track the person
Could not track the person
